In [18]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [3]:
df = pd.read_csv('Train.csv')

df.head()

,id_code,current_date,current_time,source_name,destination_name,train_name,target,country_code_source,longitude_source,latitude_source,mean_halt_times_source,country_code_destination,longitude_destination,latitude_destination,mean_halt_times_destination,current_year,current_week,current_day,is_weekend
0,isfywypmkqqhyft,2016-07-27,08:05:51 PM,station$147,station$1,ICZVZS,high,whber,4.356801,50.845658,634.16474,NaN,NaN,NaN,NaN,2016,30,Wednesday,False
1,mqsfxyvuqpbwomk,2016-07-27,08:06:11 PM,station$147,station$1,ICZVZS,high,whber,4.356801,50.845658,634.16474,NaN,NaN,NaN,NaN,2016,30,Wednesday,False
2,alspwwtbdvqsgby,2016-07-27,08:08:57 PM,station$147,station$1,ICZVZS,high,whber,4.356801,50.845658,634.16474,NaN,NaN,NaN,NaN,2016,30,Wednesday,False
3,szitxhhqduyrqpg,2016-07-27,08:09:08 PM,station$147,station$1,ICZVZS,high,whber,4.356801,50.845658,634.16474,NaN,NaN,NaN,NaN,2016,30,Wednesday,False
4,krisdqzczivvwcp,2016-07-27,08:11:01 PM,station$147,station$1,ICZVZS,high,whber,4.356801,50.845658,634.16474,NaN,NaN,NaN,NaN,2016,30,Wednesday,False


In [4]:
column_x = list(df.columns)
column_x.remove('target')
column_y = 'target'
print(column_x)
print(column_y)

['id_code', 'current_date', 'current_time', 'source_name', 'destination_name', 'train_name', 'country_code_source', 'longitude_source', 'latitude_source', 'mean_halt_times_source', 'country_code_destination', 'longitude_destination', 'latitude_destination', 'mean_halt_times_destination', 'current_year', 'current_week', 'current_day', 'is_weekend']
target


In [7]:
df_train,df_test = train_test_split(df,test_size=0.25)
df_train_x = df_train[column_x]
df_train_y = df_train[column_y]
df_test_x = df_test[column_x]
df_test_y = df_test[column_y]
print(df_train_x.shape)
print(df_test_x.shape)

(963, 18)
(321, 18)


In [8]:
df_train_x.nunique()

id_code                        963
current_date                    60
current_time                   946
source_name                    167
destination_name               193
train_name                     477
country_code_source              3
longitude_source               166
latitude_source                166
mean_halt_times_source         141
country_code_destination         3
longitude_destination          191
latitude_destination           190
mean_halt_times_destination    160
current_year                     1
current_week                    11
current_day                      7
is_weekend                       2
dtype: int64

In [9]:
missing_cols = [col for col in df_train_x.columns if df_train_x[col].isnull().any()]
missing_cols

['country_code_source',
 'longitude_source',
 'latitude_source',
 'mean_halt_times_source',
 'country_code_destination',
 'longitude_destination',
 'latitude_destination',
 'mean_halt_times_destination']

## Data Description.
1. We can ignore the **id** and **name** of the train, drop these columns.
2. **coutry_code_source** and **current_day**  have catagorical data so let's one-hot encode these.
3. will use imputation to handle missing values.

### Handling missing values.

In [10]:
def handle_missing(df):
    my_imputer = SimpleImputer(strategy='most_frequent')
    for col in missing_cols:
        df[col + '_was_missing'] = df[col].isnull()
    cols = df.columns
    df = pd.DataFrame(my_imputer.fit_transform(df))
    df.columns = cols
    return df
    
df_train_x = handle_missing(df_train_x)
df_train_x.head()

,id_code,current_date,current_time,source_name,destination_name,train_name,country_code_source,longitude_source,latitude_source,mean_halt_times_source,...,current_day,is_weekend,country_code_source_was_missing,longitude_source_was_missing,latitude_source_was_missing,mean_halt_times_source_was_missing,country_code_destination_was_missing,longitude_destination_was_missing,latitude_destination_was_missing,mean_halt_times_destination_was_missing
0,tfvyofocdqyzocu,2016-09-17,10:01:08 PM,station$525,station$150,ICZVAS,whber,3.21673,51.1972,164.419,...,Saturday,True,False,False,False,False,False,False,False,False
1,tdfxkesazkbkvuo,2016-09-23,07:04:22 AM,station$185,station$259,ICWXYW,whber,4.43222,51.1992,421.645,...,Friday,False,False,False,False,False,False,False,False,False
2,vuzylxyzfdkhrdi,2016-09-23,07:02:17 PM,station$281,station$312,ICYWWA,whber,5.56669,50.6245,269.124,...,Friday,False,False,False,False,False,False,False,False,False
3,tiyyapgsykcsvyw,2016-09-29,04:24:21 PM,station$115,station$116,SSUVUV,whber,4.53286,50.7114,30.7341,...,Thursday,False,False,False,False,False,False,False,False,False
4,jhskynnkfqffhkr,2016-09-18,07:40:09 PM,station$544,station$525,ICZSWY,whber,3.71068,51.0359,309.014,...,Sunday,True,False,False,False,False,False,False,False,False


In [11]:
df_train_x.describe(include='all')

,id_code,current_date,current_time,source_name,destination_name,train_name,country_code_source,longitude_source,latitude_source,mean_halt_times_source,...,current_day,is_weekend,country_code_source_was_missing,longitude_source_was_missing,latitude_source_was_missing,mean_halt_times_source_was_missing,country_code_destination_was_missing,longitude_destination_was_missing,latitude_destination_was_missing,mean_halt_times_destination_was_missing
count,963,963,963,963,963,963,963,963.000000,963.000000,963.000000,...,963,963,963,963,963,963,963,963,963,963
unique,963,60,946,167,193,477,3,166.000000,166.000000,141.000000,...,7,2,2,2,2,2,2,2,2,2
top,midubrweuhfcwsj,2016-09-29,01:20:21 PM,station$130,station$150,ICZVZS,whber,4.360846,50.859663,640.265896,...,Thursday,False,False,False,False,False,False,False,False,False
freq,1,74,2,85,73,22,961,86.000000,86.000000,86.000000,...,200,821,962,962,962,962,943,943,943,943


### Selecting columns to drop.
1. drop **id_code** , **current_date** (as we have other columns determining date), **dropping source and destination name** (lat, long, and coutry source code.) 

In [12]:


def drop_missing_time_handle(df):
    time = df['current_time'].str.split(n=1, expand = True)
    time_line = time[0].str.split(':',n=2, expand=True)
    df['DST'] = time[1]
    df['Hr'] = time_line[0]
    df['Min'] = time_line[1]
    df['Sec'] = time_line[2]
    df.drop(['id_code','train_name','current_date','current_time','source_name','destination_name','country_code_source','country_code_destination'],axis=1,inplace=True)
    
    return df

df_train_x = drop_missing_time_handle(df_train_x)
df_train_x.head()

,longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination,current_year,current_week,current_day,is_weekend,...,latitude_source_was_missing,mean_halt_times_source_was_missing,country_code_destination_was_missing,longitude_destination_was_missing,latitude_destination_was_missing,mean_halt_times_destination_was_missing,DST,Hr,Min,Sec
0,3.21673,51.1972,164.419,4.33653,50.8357,686.616,2016,37,Saturday,True,...,False,False,False,False,False,False,PM,10,01,08
1,4.43222,51.1992,421.645,5.11634,51.191,59.7514,2016,38,Friday,False,...,False,False,False,False,False,False,AM,07,04,22
2,5.56669,50.6245,269.124,5.45766,50.5953,56.1908,2016,38,Friday,False,...,False,False,False,False,False,False,PM,07,02,17
3,4.53286,50.7114,30.7341,4.54915,50.6938,18.2832,2016,39,Thursday,False,...,False,False,False,False,False,False,PM,04,24,21
4,3.71068,51.0359,309.014,3.21673,51.1972,164.419,2016,37,Sunday,True,...,False,False,False,False,False,False,PM,07,40,09


### Handling Catagorical data

In [13]:
df_train_x.dtypes

longitude_source                           object
latitude_source                            object
mean_halt_times_source                     object
longitude_destination                      object
latitude_destination                       object
mean_halt_times_destination                object
current_year                               object
current_week                               object
current_day                                object
is_weekend                                 object
country_code_source_was_missing            object
longitude_source_was_missing               object
latitude_source_was_missing                object
mean_halt_times_source_was_missing         object
country_code_destination_was_missing       object
longitude_destination_was_missing          object
latitude_destination_was_missing           object
mean_halt_times_destination_was_missing    object
DST                                        object
Hr                                         object


In [14]:
catagorical_cols = ['current_day','DST']
catagorical_cols

['current_day', 'DST']

In [15]:


def one_hot_encode(df):
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_df = pd.DataFrame(OH_encoder.fit_transform(df[catagorical_cols]))
    OH_df.index = df.index
    df.drop(catagorical_cols, axis=1, inplace=True)
    df = pd.concat([df,OH_df], axis=1)
    
    return df

df_train_x = one_hot_encode(df_train_x)
df_train_x

,longitude_source,latitude_source,mean_halt_times_source,longitude_destination,latitude_destination,mean_halt_times_destination,current_year,current_week,is_weekend,country_code_source_was_missing,...,Sec,0,1,2,3,4,5,6,7,8
0,3.21673,51.1972,164.419,4.33653,50.8357,686.616,2016,37,True,False,...,08,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,4.43222,51.1992,421.645,5.11634,51.191,59.7514,2016,38,False,False,...,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.56669,50.6245,269.124,5.45766,50.5953,56.1908,2016,38,False,False,...,17,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.53286,50.7114,30.7341,4.54915,50.6938,18.2832,2016,39,False,False,...,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,3.71068,51.0359,309.014,3.21673,51.1972,164.419,2016,37,True,False,...,09,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,4.36085,50.8597,640.266,4.71587,50.8823,351.916,2016,39,False,False,...,27,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
959,4.36085,50.8597,640.266,4.01457,50.7237,21.737,2016,38,False,False,...,28,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
960,3.21673,51.1972,164.419,3.71068,51.0359,309.014,2016,39,False,False,...,28,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
961,4.3568,50.8457,634.165,4.36085,50.8597,640.266,2016,32,False,False,...,05,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


#### Prepare Validation data

In [17]:
df_test_x = handle_missing(df_test_x)
df_test_x = drop_missing_time_handle(df_test_x)
df_test_x = one_hot_encode(df_test_x)
print(df_test_x.shape)
print(df_train_x.shape)

(321, 29)
(963, 29)


### Apply Machine learning algorithms 